# Robotics, Vision & Control 3e: for Python
## Chapter 9: Dynamics and Control

In [ ]:
try:
    from google.colab import output
    print('Running on CoLab')
    output.enable_custom_widget_manager()
    !pip install ipympl
    !pip install roboticstoolbox-python>=1.0.2
    !pip install --no-deps rvc3python
    COLAB = True

except ModuleNotFoundError:
    COLAB = False

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"
from IPython.display import HTML

import matplotlib.pyplot as plt

# add RTB examples folder to the path
import sys, os.path
import RVC3 as rvc
sys.path.append(os.path.join(rvc.__path__[0], 'models'))

# helper function to run bdsim in a subprocess and transfer results using a pickle file
import pickle
def run_shell(tool, **params):
    global out
    pyfile = os.path.join(rvc.__path__[0], "models", tool+".py")
    cmd = f"python {pyfile} -H +a -o"
    for key, value in params.items():
        cmd += f' --global "{key}={value}"'
    print(cmd)
    os.system(cmd)
    with open("bd.out", "rb") as f:
        out = pickle.load(f)

# ------ standard imports ------ #
import numpy as np
import math
from math import pi
np.set_printoptions(
    linewidth=120, formatter={
        'float': lambda x: f"{0:8.4g}" if abs(x) < 1e-10 else f"{x:8.4g}"})
np.random.seed(0)
from spatialmath import *
from spatialmath.base import *
from roboticstoolbox import *

# 9.1 Independent Joint Control


## 9.1.1 Actuators


## 9.1.2 Friction


## 9.1.3 Effect of the Link Mass


In [ ]:
import sympy
a1, a2, r1, r2, m1, m2, g = sympy.symbols("a1 a2 r1 r2 m1 m2 g")
link1 = Link(ET.Ry(flip=True), m=m1, r=[r1, 0, 0], name="link0")
link2 = Link(ET.tx(a1) * ET.Ry(flip=True), m=m2, r=[r2, 0, 0], name="link1")
robot = ERobot([link1, link2])

In [ ]:
robot.dynamics()

In [ ]:
q = sympy.symbols("q:2")
qd = sympy.symbols("qd:2")
qdd = sympy.symbols("qdd:2")

In [ ]:
tau = robot.rne(q, qd, qdd, gravity=[0, 0, g], symbolic=True);

## 9.1.4 Gearbox


## 9.1.5 Modeling the Robot Joint


In [ ]:
puma = models.DH.Puma560();  # load model of PUMA560 with dynamic parameters
tf = puma.jointdynamics(puma.qn);

In [ ]:
tf[1]

## 9.1.6 Velocity Control Loop


In [ ]:
if COLAB:
  %run -m vloop_test -H -g  # no graphics
else:
  run_shell("vloop_test")

## 9.1.7 Position Control Loop


In [ ]:
if COLAB:
  %run -m ploop_test -H -g  # no graphics
else:
  run_shell("ploop_test")

## 9.1.8 Independent Joint Control Summary


# 9.2 Rigid-Body Equations of Motion


In [ ]:
puma = models.DH.Puma560();

In [ ]:
zero = np.zeros((6,));
Q = puma.rne(puma.qn, zero, zero)

In [ ]:
Q = puma.rne(puma.qn, zero, zero, gravity=[0, 0, 0])

In [ ]:
traj = jtraj(puma.qz, puma.qr, 10);
Q = puma.rne(traj.q, traj.qd, traj.qdd);

In [ ]:
Q.shape

In [ ]:
Q[5, :]

In [ ]:
puma.rne(puma.qn, [1, 0, 0, 0, 0, 0], zero, gravity=[0, 0, 0])

print(puma[1].dyn())

## 9.2.1 Gravity Term


In [ ]:
Q = puma.gravload(puma.qn)

In [ ]:
puma.gravity

In [ ]:
puma.gravity /= 6

In [ ]:
puma.gravload(puma.qn)

In [ ]:
puma.base = SE3.Rx(pi);
puma.gravload(puma.qn)

In [ ]:
puma = models.DH.Puma560();

In [ ]:
Q = puma.gravload(puma.qs)

In [ ]:
Q = puma.gravload(puma.qr)

In [ ]:
N = 100;
Q1, Q2 = np.meshgrid(np.linspace(-pi, pi, N), np.linspace(-pi, pi, N));
G1, G2 = np.zeros((N,N)), np.zeros((N,N));
for i in range(N):
  for j in range(N):
    g = puma.gravload(np.array([0, Q1[i,j], Q2[i,j], 0, 0, 0]))
    G1[i, j] = g[1]  # shoulder gravity load
    G2[i, j] = g[2]  # elbow gravity load
plt.axes(projection="3d").plot_surface(Q1, Q2, G1);

## 9.2.2 Inertia Matrix


In [ ]:
M = puma.inertia(puma.qn)

In [ ]:
N = 100;
Q1, Q2 = np.meshgrid(np.linspace(-pi, pi, N), np.linspace(-pi, pi, N));
M00, M01, M11 = np.zeros((N,N)), np.zeros((N,N)), np.zeros((N,N));
for i in range(N):
  for j in range(N):
    M = puma.inertia(np.array([0, Q1[i,j], Q2[i,j], 0, 0, 0]))
    M00[i, j] = M[0, 0]
    M01[i, j] = M[0, 1]
    M11[i, j] = M[1, 1]
plt.axes(projection="3d").plot_surface(Q1, Q2, M00);

In [ ]:
M00.max() / M00.min()

## 9.2.3 Friction


In [ ]:
puma.friction([1, 0, 0, 0, 0, 0])

## 9.2.4 Coriolis and Centripetal Matrix


In [ ]:
qd = [0, 0, 1, 0, 0, 0];

In [ ]:
C = puma.coriolis(puma.qn, qd)

In [ ]:
C @ qd

## 9.2.5 Effect of Payload


In [ ]:
G = puma.gravload(puma.qn);
M = puma.inertia(puma.qn);

In [ ]:
puma.payload(2.5, [0, 0, 0.1]);

In [ ]:
M_loaded = puma.inertia(puma.qn);

In [ ]:
M_loaded / np.where(M < 1e-6, np.nan, M)

In [ ]:
puma.gravload(puma.qn) / np.where(G < 1e-6, np.nan, G)

In [ ]:
puma.payload(0)

## 9.2.6 Base Wrench


In [ ]:
Q, wb = puma.rne(puma.qn, zero, zero, base_wrench=True);

In [ ]:
wb

In [ ]:
sum([link.m for link in puma]) * puma.gravity[2]

## 9.2.7 Dynamic Manipulability


In [ ]:
Jt = puma.jacob0(puma.qn, half="trans");  # first 3 rows

In [ ]:
M = puma.inertia(puma.qn);
E = (Jt @ np.linalg.inv(M) @ np.linalg.inv(M).T @ Jt.T);
plot_ellipsoid(E);

In [ ]:
e, _ = np.linalg.eig(E)
radii = 1 / np.sqrt(e)

In [ ]:
radii.min() / radii.max()

In [ ]:
puma.manipulability(puma.qn, method="asada")

# 9.3 Forward Dynamics


In [ ]:
qdd = puma.accel(puma.q, puma.qd, Q)

In [ ]:
if COLAB:
  %run -m zerotorque -H -g  # no graphics
else:
  run_shell("zerotorque")

In [ ]:
xplot(out.t, out.x[:, :3])

In [ ]:
# note the next line is wrong in the book, there is an extra passed parameter 'robot'
torque_func = lambda robot, t, q, qd: np.zeros((6,))
traj = puma.nofriction().fdyn(T=5, q0=puma.qr, Q=torque_func)
xplot(traj.t, traj.q)

In [ ]:
puma_nf = puma.nofriction();

# 9.4 Rigid-Body Dynamics Compensation
#

## 9.4.1 Feedforward Control


In [ ]:
if COLAB:
  %run -m feedforward -H -g  # no graphics
else:
  run_shell("bdrun feedforward")

## 9.4.2 Computed-Torque Control


In [ ]:
if COLAB:
  %run -m computed-torque-main -H -g  # no graphics
else:
  run_shell("computed-torque-main")

# 9.5 Task-Space Dynamics and Control


In [ ]:
puma = models.DH.Puma560();

xd = [0, 0.1, 0, 0, 0, 0];
qd = np.linalg.inv(puma.jacob0_analytical(puma.qn, "eul")) @ xd;
Cx = puma.coriolis_x(puma.qn, qd, representation="eul")

In [ ]:
Cx @ xd

In [ ]:
Mx = puma.inertia_x(puma.qn, representation="eul")

In [ ]:
np.linalg.inv(Mx) @ [10, 0, 0, 0, 0, 0]

# 9.6 Applications


## 9.6.1 Operational Space Control


In [ ]:
if COLAB:
  %run -m opspace -H -g  # no graphics
else:
  run_shell("opspace")

## 9.6.2 Series-Elastic Actuator (SEA)


In [ ]:
if COLAB:
  %run -m SEA -H -g  # no graphics
else:
  run_shell("SEA")